In [29]:
import tweepy

In [ ]:
from textblob import TextBlob

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = ''
        consumer_secret = ''
        access_token = ''
        access_token_secret = ''
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
            #data = self.api.rate_limit_status()
            print("Authentication OK")
            
        except:
            print("Error: Authentication Failed")
            
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
    def get_tweets(self, query, count, page, start, end):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            #fetched_tweets = self.api.search(q = query, rpp = count) 
            for tweet in tweepy.Cursor(self.api.search, q=query, until=end, lang="en").items(count):
                print(tweet.text)
                cleaned_tweet = TextBlob(self.clean_tweet(tweet.text))
                print(cleaned_tweet)
                
        except tweepy.TweepError as e:
            print("Error : " + str(e))
    
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    while True:
        api.get_tweets(query = "Microsoft", count=1000, page = 1, start=datetime.date.today()-datetime.timedelta(days=30), end=datetime.date.today())

    
if __name__ == "__main__":
    # calling main function
    main()

Authentication OK
Microsoft lays off &gt;50 journalists replacing them with AI
https://t.co/6S3OSkLCGj
Microsoft lays off gt 50 journalists replacing them with AI
Microsoft demobilizes journalists and uses “Android” services https://t.co/xC2jABI8MP
Microsoft demobilizes journalists and uses Android services
Luciferase 060606. Maria Abramovic met with Jacob Rothschild. SRS 6++ Cannibals of Dead, Rothschild. https://t.co/KrVQKleXce.
Luciferase 060606 Maria Abramovic met with Jacob Rothschild SRS 6 Cannibals of Dead Rothschild
RT @MhdAizat__: For all student out there, if you have a group assignment, please use google docs. You guys may search it on google. It is…
RT For all student out there if you have a group assignment please use google docs You guys may search it on google It is
RT @vanguardngrnews: Microsoft lays off journalists to replace them with AI https://t.co/a3lxKkVM0v #vanguardnews https://t.co/2f4vQUFF9t
RT Microsoft lays off journalists to replace them with AI vanguardnews

RT @engadget: Microsoft cuts dozens of staff as it shifts to AI for MSN news stories https://t.co/Mv9shz8w26 https://t.co/XmYoPBmde2
RT Microsoft cuts dozens of staff as it shifts to AI for MSN news stories
@BradOrluk @CheckPointSW @Microsoft Excited for you man!!
Excited for you man
@McDaddyTree 1/2 Hi there! Thanks for bringing this to our attention. We'd like to look into this for you. Is this… https://t.co/4ijTQgsYXB
1 2 Hi there Thanks for bringing this to our attention We d like to look into this for you Is this
RT @engadget: Microsoft cuts dozens of staff as it shifts to AI for MSN news stories https://t.co/Mv9shz8w26 https://t.co/XmYoPBmde2
RT Microsoft cuts dozens of staff as it shifts to AI for MSN news stories
@Kay99699491 the Wal-Mart employee said i could. he said you can play nintendo sony Microsoft and sega games on the Nintendo
the Wal Mart employee said i could he said you can play nintendo sony Microsoft and sega games on the Nintendo
Microsoft lays off journalists to

RT @GowDeclassified: ICYMI: The Xbox Series X doubles down on backwards compatibility as Microsoft reveals key innovative features.

What d…
RT ICYMI The Xbox Series X doubles down on backwards compatibility as Microsoft reveals key innovative features What d
Microsoft lays off journalists to replace them with AI https://t.co/IEbJ2Aottq
Microsoft lays off journalists to replace them with AI
RT @learnk8s: The first Kubernetes attack matrix comprising the major techniques that are relevant to container orchestration security, wit…
RT The first Kubernetes attack matrix comprising the major techniques that are relevant to container orchestration security wit
"Microsoft will Replace Journalists with AI "https://t.co/3HnlvSr1GO by @Mike_K_Spencer on @LinkedIn
Microsoft will Replace Journalists with AI by K Spencer on
@CAMELOT331 @Xbox @Sony But I think Microsoft is going to probably have the best multiples &amp; sell the most (minus t… https://t.co/xboszU50sT
But I think Microsoft is going t

Reverb G2: a new high-resolution VR headset from HP, Microsoft and Valve https://t.co/Hf4OEBvLxi
Reverb G2 a new high resolution VR headset from HP Microsoft and Valve
RT @RexOmarrr: Dear Microsoft, 

Kindly do same in Ghana for us. We have assembled slay queens, you and the police journalist, militia gene…
RT Dear Microsoft Kindly do same in Ghana for us We have assembled slay queens you and the police journalist militia gene
RT @RexOmarrr: Dear Microsoft, 

Kindly do same in Ghana for us. We have assembled slay queens, you and the police journalist, militia gene…
RT Dear Microsoft Kindly do same in Ghana for us We have assembled slay queens you and the police journalist militia gene
Microsoft 'to replace journalists with robots' https://t.co/4EhL0sEWdd show me the names of your authors on your… https://t.co/rAwCObIlrH
Microsoft to replace journalists with robots show me the names of your authors on your
RT @ErgoDemise: More menus await us tonight in #PhantasyStarOnline2! Remember thi

@Microsoft Too close to the screens. Bad for eyes and back.
Too close to the screens Bad for eyes and back
RT @NintenZ: So rumor has it SEGA is reentering the console market

...in Japan only

Makes sense to me honestly, they’ve worked closely wi…
RT So rumor has it SEGA is reentering the console market in Japan only Makes sense to me honestly they ve worked closely wi
Microsoft cuts dozens of staff as it shifts to AI for MSN news stories https://t.co/SrDni3AXE0 https://t.co/iMPrnM8hhg
Microsoft cuts dozens of staff as it shifts to AI for MSN news stories
RT @newsycombinator: Microsoft 'To Replace Journalists with Robots' https://t.co/DwHyzQZIiy
RT Microsoft To Replace Journalists with Robots
RT @jdbnet: Microsoft responsible machine learning capabilities build trust in AI systems, developers say https://t.co/HZEPtxtgQ4 #ai
RT Microsoft responsible machine learning capabilities build trust in AI systems developers say ai
@BanditGrinds As long as it’s Microsoft, it will be easier to hac

...

Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429


Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429


Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429


Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429


Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
Error : Twitter error response: status code = 429
